In [1]:
import sys
sys.path.append('..')
sys.path.append('../..')

from sklearn.model_selection import train_test_split

import tensorflow.compat.v1 as tf

import numpy as np

import utils.losses_utils as losses

import utils.data_utils as data_utils

import utils.viz_utils as viz

import matplotlib.pyplot as plt


bold_shift=3
f_resample=1.8
n_partitions=25
activation=''

eeg_train, bold_train, eeg_val, bold_val = data_utils.load_data(list(range(10)), 
                                                                list(range(10, 12)), 
                                                                bold_shift=bold_shift, 
                                                                n_partitions=n_partitions, f_resample=f_resample,
                                                                roi=1, roi_ica_components=20)

#standardize data
eeg_train, bold_train, eeg_scaler, bold_scaler = data_utils.standardize(eeg_train, bold_train)
eeg_val, bold_val, _, _ = data_utils.standardize(eeg_val, bold_val, eeg_scaler=eeg_scaler, bold_scaler=bold_scaler)

n_voxels = bold_train.shape[1]

print("Finished Loading Data")

X_train_eeg, X_train_bold, tr_y = data_utils.create_eeg_bold_pairs(eeg_train, bold_train)
X_val_eeg, X_val_bold, tv_y = data_utils.create_eeg_bold_pairs(eeg_val, bold_val)


X_train_eeg = X_train_eeg.astype(np.float32)
X_train_bold = X_train_bold.astype(np.float32)
X_val_eeg = X_val_eeg.astype(np.float32)
X_val_bold = X_val_bold.astype(np.float32)


tr_y = np.array(tr_y, dtype=np.float32)
tv_y = np.array(tv_y, dtype=np.float32)

eeg_train = eeg_train.astype('float32')
bold_train = bold_train.astype('float32')
eeg_val = eeg_val.astype('float32')
bold_val = bold_val.astype('float32')

print("Pairs Created")

_, _, eeg_test, bold_test = data_utils.load_data(list(range(0)), list(range(12, 16)), 
                                                 bold_shift=bold_shift, 
                                                 n_partitions=n_partitions, 
                                                 f_resample=f_resample, 
                                                 roi=1, roi_ica_components=20)

eeg_test, bold_test, _, _ = data_utils.standardize(eeg_test, bold_test, eeg_scaler=eeg_scaler, bold_scaler=bold_scaler)

eeg_test = eeg_test.astype('float32')
bold_test = bold_test.astype('float32')

/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/davidcalhas/anacond

Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/32/export/20130410320002_Segmentation_bin.vhdr...
Setting channel info structure...
Reading 0 ... 162022  =      0.000 ...   648.088 secs...
(50, 2607, 7)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/35/export/20130424350002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 197234  =      0.000 ...   788.936 secs...
(100, 2607, 7)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/36/export/20130425360002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 181949  =      0.000 ...   727.796 secs...
(150, 2607, 7)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/37/export/20130426370002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 195159  =      0.000 ...   780.636 secs...
(200, 2607, 7)
Extracting parameters from /home/davidcalhas/e

In [289]:
eeg_model = tf.keras.Sequential([tf.keras.layers.ConvLSTM2D(1, kernel_size=(5, 64), strides=(1, 1), activation='sigmoid', recurrent_activation='sigmoid', unit_forget_bias=True, return_sequences=True),
                                tf.keras.layers.Reshape((14, 1))])

bold_model = tf.keras.Sequential([tf.keras.layers.Conv2D(1, kernel_size=(2607, 1), strides=(1)),
                                tf.keras.layers.Reshape((14, 1)),
                                 tf.keras.layers.LSTM(1, return_sequences=True)])

In [290]:
#X_train_eeg = X_train_eeg.reshape(X_train_eeg.shape[0], X_train_eeg.shape[3], X_train_eeg.shape[2], X_train_eeg.shape[1], X_train_eeg.shape[4])
#X_val_eeg = X_val_eeg.reshape(X_val_eeg.shape[0], X_val_eeg.shape[3], X_val_eeg.shape[2], X_val_eeg.shape[1], X_val_eeg.shape[4])

input_eeg = tf.keras.Input(X_train_eeg.shape[1:])
processed_eeg = eeg_model(input_eeg)


input_bold = tf.keras.Input(X_train_bold.shape[1:])
processed_bold = bold_model(input_bold)

correlation = tf.keras.layers.Lambda(losses.correlation_angle, 
                                     output_shape=losses.cos_dist_output_shape, 
                                     name="correlation_layer")([processed_eeg, processed_bold])

multi_modal_model = tf.keras.Model([input_eeg, input_bold], correlation)

In [291]:
loss=losses.contrastive_loss
#optimizer = tf.keras.optimizers.Adam(lr=0.0001)#, clipvalue=0.1)
optimizer = tf.keras.optimizers.RMSprop(lr=0.0001, rho=0.9, clipvalue=0.1)
epochs=500
batch_size=16

class custom_training_loss:
    def __init__(self):
        self.encoder_loss = 0
        self.decoder_loss = 0
        self.batch = 0
        
    def update_batch_decoder_loss_avg(self, loss):
        self.decoder_loss += loss
        self.batch += 1
    
    def get_batch_decoder_loss_avg(self):
        return self.decoder_loss/self.batch
    
    def update_batch_encoder_loss_avg(self, loss):
        self.encoder_loss += loss
        self.batch += 1
    
    def get_batch_encoder_loss_avg(self):
        return self.encoder_loss/self.batch
    
def grad_multi_encoder(model, inputs, targets):
    eeg = tf.convert_to_tensor(inputs[0])
    bold = tf.convert_to_tensor(inputs[1])
    with tf.GradientTape() as tape:
        #tape.watch(model.variables)
        tape.watch(eeg)
        tape.watch(bold)
        outputs = model([eeg, bold])
        
        encoder_loss = losses.contrastive_loss(outputs, targets)
        
        return encoder_loss,  tape.gradient(encoder_loss, 
                                            model.trainable_weights)

In [292]:
for epoch in range(epochs):

    _losses = custom_training_loss()

    for batch_init in range(0, len(X_train_eeg), batch_size):
        batch_start = batch_init
        if(batch_start + batch_size >= len(X_train_eeg)):
            batch_stop = len(X_train_eeg)
        else:
            batch_stop = batch_start + batch_size

        #now train the compression by correlation model
        encoder_loss, encoder_grads = grad_multi_encoder(multi_modal_model, 
                                                         [X_train_eeg[batch_start:batch_stop], 
                                                                  X_train_bold[batch_start:batch_stop]], 
                                                         tr_y[batch_start:batch_stop])
        
        #encoder_grads, _ = tf.clip_by_global_norm(encoder_grads, 0.1)
        
        with tf.name_scope("gradient_encoders") as scope:
            optimizer.apply_gradients(zip(encoder_grads, multi_modal_model.trainable_variables), name=scope)

        # Track progress
        _losses.update_batch_encoder_loss_avg(encoder_loss)
        
    # end epoch
    encoder_loss = _losses.get_batch_encoder_loss_avg()

    print("Encoder Loss: ", tf.keras.backend.eval(encoder_loss), 
          "Validation Encoder Loss: ", losses.contrastive_loss(multi_modal_model([X_val_eeg, X_val_bold]),tv_y))

Encoder Loss:  0.074091315 Validation Encoder Loss:  tf.Tensor(0.38848683, shape=(), dtype=float32)
Encoder Loss:  0.06875455 Validation Encoder Loss:  tf.Tensor(0.3390912, shape=(), dtype=float32)
Encoder Loss:  0.06738542 Validation Encoder Loss:  tf.Tensor(0.33787605, shape=(), dtype=float32)
Encoder Loss:  0.06698361 Validation Encoder Loss:  tf.Tensor(0.33778113, shape=(), dtype=float32)
Encoder Loss:  0.06683948 Validation Encoder Loss:  tf.Tensor(0.3462588, shape=(), dtype=float32)
Encoder Loss:  0.066780925 Validation Encoder Loss:  tf.Tensor(0.33776736, shape=(), dtype=float32)
Encoder Loss:  0.06674066 Validation Encoder Loss:  tf.Tensor(0.33973122, shape=(), dtype=float32)
Encoder Loss:  0.06672839 Validation Encoder Loss:  tf.Tensor(0.33598655, shape=(), dtype=float32)
Encoder Loss:  0.06671071 Validation Encoder Loss:  tf.Tensor(0.33482206, shape=(), dtype=float32)
Encoder Loss:  0.066699594 Validation Encoder Loss:  tf.Tensor(0.3347149, shape=(), dtype=float32)
Encoder Lo

KeyboardInterrupt: 

In [284]:
losses.contrastive_loss(multi_modal_model([X_val_eeg, X_val_bold]),tv_y)

<tf.Tensor: id=122509649, shape=(), dtype=float32, numpy=0.3983344>